# * TOL Analysis

## Import Data Source

In [63]:
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from functools import reduce
import datetime

mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

In [64]:
as_week = 2023027
data_src = f"../../data/raw/Raw-Agg Perf Area_Wk {as_week}.csv"
df = pd.read_csv(data_src, low_memory=False)
cols = ['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'CENTER', 'METRIC_GRP', 'COMP_CD', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_AS_OF', 'AGG_TYPE', 'RR_IND', 'GRY_IND', 'UOM', 'PERIOD', 'ACTUAL_SNAP', 'ACTUAL_AGG', 'TARGET_SNAP', 'TARGET_AGG', 'PPN_TM']
df = df[cols]

''' Fix Metric CD List '''
tol_analysis_cd_list = ['TB3R000100', 'TB3R000101', 'TB3R000102', # "TOL Revenue", "TOL New Revenue", "TOL Exisitng Revenue"
                        'TB3S000100', 'TB3S000101', 'TB3S000400', 'TB3S000500', # "TOL Connected Subs", "TOL Registration Subs", "TOL Net Add", "TOL Active Subs"
                        'TSER13100', 'TB3R000600'] # "TOL Churn Subs", "TOL Connected 1MTH RC"

In [65]:
df.shape

(135720, 26)

## Data Preporation

In [66]:
''' Functions '''

def unit_mb(baht):
    n = baht
    unit_mb = n / 1000000
    return unit_mb

def unit_k(baht):
    n = baht
    unit_k = n / 1000
    return unit_k

In [67]:
''' Rawdata '''

''' Filters '''
df = df[df['TM_KEY_MTH'] <= 202306] # MONTH filter
# df = df[df['TM_KEY_WK'] <= 2023022] # WEEK filter
# df = df[df['TM_KEY_DAY'] <= 20230630] # DAY filter
df = df[df['METRIC_CD'].isin(tol_analysis_cd_list)]

''' Modify '''
df = df.rename(columns={'ACTUAL_SNAP':'ACT_SNAP', 'ACTUAL_AGG':'ACT_AGG', 'TARGET_SNAP':'TGT_SNAP', 'TARGET_AGG':'TGT_AGG'})

''' Add new column '''
df['MTH_DESC'] = pd.to_datetime(df['TM_KEY_MTH'], format='%Y%m', errors='coerce').dt.strftime('%B')
df['WK_LABEL'] = df['TM_KEY_WK'].apply(str).str[5:].astype(int)
df['DAY_LABEL'] = df['TM_KEY_DAY'].apply(str).str[6:].astype(int)

df.tail(3)

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,METRIC_GRP,COMP_CD,PRODUCT_GRP,METRIC_CD,...,UOM,PERIOD,ACT_SNAP,ACT_AGG,TGT_SNAP,TGT_AGG,PPN_TM,MTH_DESC,WK_LABEL,DAY_LABEL
134789,20230630,2023026,202306,20232,2023,Sales & Revenue,Revenue,True,TOL,TB3R000100,...,baht,M,NaN,7.171993e+06,NaN,7.544500e+06,2023-07-09 22:31:55,June,26,30
134790,20230630,2023026,202306,20232,2023,Sales & Revenue,Revenue,True,TOL,TB3R000100,...,baht,M,NaN,5.366862e+06,NaN,5.338382e+06,2023-07-09 22:31:55,June,26,30
134791,20230630,2023026,202306,20232,2023,Sales & Revenue,Revenue,True,TOL,TB3R000100,...,baht,M,NaN,2.873814e+08,NaN,3.164465e+08,2023-07-09 22:31:55,June,26,30


In [68]:
''' Create Dataframe '''

''' Aggregate type : S '''
df_agg_s = df[df['AGG_TYPE']=='S']
df_agg_s = df_agg_s.rename(columns={'ACT_SNAP':'ACT', 'TGT_SNAP':'TGT'})

''' Aggregate type : N '''
df_agg_n = df[df['AGG_TYPE']=='N'].dropna(subset='ACT_SNAP')
df_agg_n = df_agg_n.rename(columns={'ACT_SNAP':'ACT', 'TGT_SNAP':'TGT'})

In [69]:
''' Period Values '''

curr_yr = df['TM_KEY_YR'].max()
last_yr = df['TM_KEY_YR'].max() - 1
curr_mth = df['TM_KEY_MTH'].max()
curr_mth_desc = pd.to_datetime(curr_mth, format='%Y%m', errors='coerce').strftime('%B')
last_mth = int(f'{str(last_yr)}12') if str(curr_mth)[4:]=='01' else curr_mth-1
curr_wk = df['TM_KEY_WK'].max()
last_wk = df['TM_KEY_WK'].drop_duplicates().shift().max()
curr_wk_label = df['WK_LABEL'].iloc[-1]
curr_day = df['TM_KEY_DAY'].max()
last_day = df['TM_KEY_DAY'].drop_duplicates().shift().max()
curr_day_label = df['DAY_LABEL'].iloc[-1]
last_ppn_tm = df['PPN_TM'].max()

wk_day_start = int(str(df[df['TM_KEY_WK']==curr_wk]['TM_KEY_DAY'].min())[6:])
wk_day_end = int(str(df[df['TM_KEY_WK']==curr_wk]['TM_KEY_DAY'].max())[6:])

In [70]:
''' Fix Columns Dict '''

cols_dict = {'TOL Revenue':'TOTAL', 'TOL New Revenue':'NEW', 'TOL Exisitng Revenue':'EXIST', 
             'TOL Connected Subs':'CON_SUB', 'TOL Registration Subs':'REG_SUB', 'TOL Net Add':'NET_ADD', 'TOL Active Subs':'ACTV_SUB', 
             'TOL Churn Subs':'CHRN_SUB', 'TOL Connected 1MTH RC':'1MTH_RC'}

### DAY : Aggregation (base level)

In [71]:
''' Create Dataframe '''

cols_agg = ['METRIC_CD', 'METRIC_NAME', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'MTH_DESC', 'DAY_LABEL', 'AGG_TYPE', 'ACT_SNAP', 'ACT_AGG', 'TGT_SNAP', 'TGT_AGG']
df_day = df[cols_agg].reset_index()

''' Aggregation '''
df_day['ACT'] = np.where(df_day['AGG_TYPE']=='N', df_day['ACT_AGG'], df_day['ACT_SNAP'])
df_day['TGT'] = np.where(df_day['AGG_TYPE']=='N', df_day['TGT_AGG'], df_day['TGT_SNAP'])
df_day['%ACHV'] = df_day['ACT'] / df_day['TGT']
df_day['GAP'] = df_day['ACT'] - df_day['TGT']
# df_day['LAST_ACT'] = df_day.groupby(['METRIC_CD', 'AREA_CD'])['ACT'].shift()
# df_day['DIFF'] = df_day['ACT'] - df_day['LAST_ACT']
# df_day['%DOD'] = (df_day['ACT'] / df_day['LAST_ACT']) - 1
df_day.tail()

# ''' Get results '''
# df_day_rev = df_day[df_day['METRIC_NAME']=='TOL Revenue'].sort_values(['TM_KEY_DAY', 'METRIC_CD', 'AREA_NO', 'AREA_CD'])
# # df_day_rev = df_day[(df_day['METRIC_NAME']=='TOL Revenue') & (df_day['AREA_CD']=='043')]
# df_day_rev.to_excel('df_day_rev.xlsx', index=False)


,index,METRIC_CD,METRIC_NAME,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,...,DAY_LABEL,AGG_TYPE,ACT_SNAP,ACT_AGG,TGT_SNAP,TGT_AGG,ACT,TGT,%ACHV,GAP
127363,134787,TB3R000100,TOL Revenue,3,H,61Y,Uthai Thani - Chai Nat,20230630,2023026,202306,...,30,S,NaN,6.728970e+06,NaN,7.295989e+06,NaN,NaN,NaN,NaN
127364,134788,TB3R000100,TOL Revenue,3,H,60Z,Nakhon Sawan,20230630,2023026,202306,...,30,S,NaN,9.109056e+06,NaN,1.004147e+07,NaN,NaN,NaN,NaN
127365,134789,TB3R000100,TOL Revenue,3,H,77Z,Prachuap Khiri Khan,20230630,2023026,202306,...,30,S,NaN,7.171993e+06,NaN,7.544500e+06,NaN,NaN,NaN,NaN
127366,134790,TB3R000100,TOL Revenue,3,H,66Z,Phichit,20230630,2023026,202306,...,30,S,NaN,5.366862e+06,NaN,5.338382e+06,NaN,NaN,NaN,NaN
127367,134791,TB3R000100,TOL Revenue,2,G,GX2,Regional Management 2 (BMA-East),20230630,2023026,202306,...,30,S,NaN,2.873814e+08,NaN,3.164465e+08,NaN,NaN,NaN,NaN


In [72]:
''' Pivot by KPIS '''

df_day_agg = df_day.pivot(index=['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'MTH_DESC', 'DAY_LABEL', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME'], 
                                  columns='METRIC_NAME', 
                                  values=['ACT', 'TGT', '%ACHV', 'GAP']).rename(columns=cols_dict).reset_index()

df_day_agg#.tail(3)

# ''' Get results '''
# df_day_agg.to_excel('df_day_agg.xlsx')

TM_KEY_DAY TM_KEY_WK TM_KEY_MTH TM_KEY_QTR TM_KEY_YR MTH_DESC  \
METRIC_NAME                                                                 
0             20230301   2023009     202303      20231      2023    March   
1             20230301   2023009     202303      20231      2023    March   
2             20230301   2023009     202303      20231      2023    March   
3             20230301   2023009     202303      20231      2023    March   
4             20230301   2023009     202303      20231      2023    March   
...                ...       ...        ...        ...       ...      ...   
14147         20230630   2023026     202306      20232      2023     June   
14148         20230630   2023026     202306      20232      2023     June   
14149         20230630   2023026     202306      20232      2023     June   
14150         20230630   2023026     202306      20232      2023     June   
14151         20230630   2023026     202306      20232      2023     June   

            DAY_LABEL AREA_NO AREA_TYPE AREA_CD  ... %ACHV           GAP  \
METRIC_NAME                                      ... TOTAL      ACTV_SUB   
0                   1       1         P       P  ...   NaN -13685.258065   
1                   1       2         G     GX0  ...   NaN           NaN   
2                   1       2         G     GX1  ...   NaN -46605.080006   
3                   1       2         G     GX2  ...   NaN -79679.983146   
4                   1       2         G     GX3  ...   NaN -28771.917572   
...               ...     ...       ...     ...  ...   ...           ...   
14147              30       3         H     92Y  ...   NaN    502.200050   
14148              30       3         H     93Z  ...   NaN   2014.939710   
14149              30       3         H     94Z  ...   NaN    753.662903   
14150              30       3         H     95Z  ...   NaN    943.030376   
14151              30       3         H     96Z  ...   NaN    913.520176   

                                                                              \
METRIC_NAME CHRN_SUB       1MTH_RC     CON_SUB EXIST     NET_ADD NEW REG_SUB   
0             -244.0 -1.278840e+06 -546.014516   NaN -302.014516 NaN     NaN   
1                NaN           NaN         NaN   NaN         NaN NaN     NaN   
2             -820.0           NaN         NaN   NaN         NaN NaN     NaN   
3             -765.0           NaN         NaN   NaN         NaN NaN     NaN   
4             -798.0           NaN         NaN   NaN         NaN NaN     NaN   
...              ...           ...         ...   ...         ...  ..     ...   
14147            1.0           NaN         NaN   NaN         NaN NaN     NaN   
14148           -2.0 -4.614463e+03   -6.997667   NaN   -4.997667 NaN     NaN   
14149           -2.0 -3.973991e+03   -4.954000   NaN   -2.954000 NaN     NaN   
14150           -1.0 -3.949306e+03   -5.902667   NaN   -4.902667 NaN     NaN   
14151            0.0 -3.659113e+03   -5.418667   NaN   -5.418667 NaN     NaN   

                   
METRIC_NAME TOTAL  
0             NaN  
1             NaN  
2             NaN  
3             NaN  
4             NaN  
...           ...  
14147         NaN  
14148         NaN  
14149         NaN  
14150         NaN  
14151         NaN  

[14152 rows x 47 columns]

### MTH : Aggregation

In [73]:
''' Create Dataframe '''

df_agg_s_mth = df_agg_s.groupby(['METRIC_CD', 'METRIC_NAME', 'TM_KEY_MTH', 'MTH_DESC', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'AGG_TYPE']).agg({'ACT': 'sum', 'TGT': 'sum'}).reset_index()
df_agg_n_mth = df_agg_n.loc[df_agg_n.groupby(['TM_KEY_MTH', 'AREA_NO', 'AREA_CD'])['TM_KEY_DAY'].idxmax(), ['METRIC_CD', 'METRIC_NAME', 'TM_KEY_MTH', 'MTH_DESC', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'AGG_TYPE', 'ACT', 'TGT']]
df_mth = pd.concat([df_agg_s_mth, df_agg_n_mth])

# ''' ARPU '''
# df_tmp_rev = df_mth[df_mth['METRIC_NAME']=='TOL Revenue'][['TM_KEY_MTH', 'MTH_DESC', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACT']]
# df_tmp_rev = df_tmp_rev.rename(columns={'ACT':'REV'})
# df_tmp_actv = df_mth[df_mth['METRIC_NAME']=='TOL Active Subs'][['TM_KEY_MTH', 'AREA_CD', 'ACT']]
# df_tmp_actv = df_tmp_actv.rename(columns={'ACT':'ACTV_SUB'})
# df_arpu = df_tmp_rev.merge(df_tmp_actv, on=['TM_KEY_MTH', 'AREA_CD'])
# df_arpu['METRIC_CD'] = 'TMP01'
# df_arpu['METRIC_NAME'] = 'ARPU'
# df_arpu['ACT'] = df_arpu['REV'] / df_arpu['ACTV_SUB'] # ARPU
# df_arpu = df_arpu[['METRIC_CD', 'METRIC_NAME', 'TM_KEY_MTH', 'MTH_DESC', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACT']]
# df_mth = pd.concat([df_mth, df_arpu])

df_mth = df_mth[df_mth['AREA_TYPE']=='H']

''' Aggregation '''
df_mth['%ACHV'] = np.where(df_mth['METRIC_NAME']=='TOL Churn Subs', -((df_mth['ACT']/df_mth['TGT'])-1), df_mth['ACT']/df_mth['TGT'])
df_mth['GAP'] = df_mth['ACT'] - df_mth['TGT']
df_mth['LAST_ACT'] = df_mth.groupby(['METRIC_CD', 'AREA_CD'])['ACT'].shift()
df_mth['DIFF'] = df_mth['ACT'] - df_mth['LAST_ACT']
df_mth['%MOM'] = (df_mth['ACT'] / df_mth['LAST_ACT']) - 1
# df_mth['ARPU'] = np.where(df_mth['METRIC_NAME']=='TOL Revenue', df_mth['ACT'], 0)
# df_mth['ARPU'] = df_mth.groupby(['TM_KEY_MTH', 'AREA_CD'])['ACT'].sum()

df_mth.tail()
# df_mth[(df_mth['METRIC_NAME']=='TOL Revenue') & (df_mth['AREA_TYPE']=='H') & (df_mth['AREA_CD']=='001')]


,METRIC_CD,METRIC_NAME,TM_KEY_MTH,MTH_DESC,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,AGG_TYPE,ACT,TGT,%ACHV,GAP,LAST_ACT,DIFF,%MOM
14148,TB3S000500,TOL Active Subs,202306,June,3,H,92Y,Trang - Satun,N,14629.0,14126.799950,1.035549,502.200050,14707.0,-78.0,-0.005304
14147,TB3S000500,TOL Active Subs,202306,June,3,H,93Z,Phatthalung,N,12942.0,10927.060290,1.184399,2014.939710,13045.0,-103.0,-0.007896
14081,TB3S000500,TOL Active Subs,202306,June,3,H,94Z,Pattani,N,8330.0,7576.337097,1.099476,753.662903,8427.0,-97.0,-0.011511
14149,TB3S000500,TOL Active Subs,202306,June,3,H,95Z,Yala,N,7588.0,6644.969624,1.141916,943.030376,7650.0,-62.0,-0.008105
14084,TB3S000500,TOL Active Subs,202306,June,3,H,96Z,Narathiwat,N,8301.0,7387.479824,1.123658,913.520176,8390.0,-89.0,-0.010608


In [74]:
''' Pivot by KPIS '''

df_mth_agg = df_mth.pivot(index=['TM_KEY_MTH', 'MTH_DESC', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME'], 
                                  columns='METRIC_NAME', 
                                  values=['ACT', 'TGT', '%ACHV', 'GAP', 'LAST_ACT', 'DIFF', '%MOM']).rename(columns=cols_dict).reset_index()

df_mth_agg['ARPU'] = df_mth_agg['ACT']['TOTAL'] / df_mth_agg['ACT']['ACTV_SUB']

df_mth_agg.tail()

TM_KEY_MTH MTH_DESC AREA_NO AREA_TYPE AREA_CD      AREA_NAME  \
METRIC_NAME                                                                
415             202306     June       3         H     92Y  Trang - Satun   
416             202306     June       3         H     93Z    Phatthalung   
417             202306     June       3         H     94Z        Pattani   
418             202306     June       3         H     95Z           Yala   
419             202306     June       3         H     96Z     Narathiwat   

                 ACT                           ...      %MOM            \
METRIC_NAME ACTV_SUB CHRN_SUB 1MTH_RC CON_SUB  ...  ACTV_SUB  CHRN_SUB   
415          14629.0    352.0   106.0   106.0  ... -0.005304 -0.043478   
416          12942.0    281.0   151.0   151.0  ... -0.007896  0.052434   
417           8330.0    218.0    71.0    71.0  ... -0.011511 -0.099174   
418           7588.0    168.0    92.0    92.0  ... -0.008105 -0.096774   
419           8301.0    224.0    97.0    97.0  ... -0.010608 -0.167286   

                                                                         \
METRIC_NAME   1MTH_RC   CON_SUB     EXIST   NET_ADD       NEW   REG_SUB   
415         -0.337500 -0.337500 -0.122165  0.159420 -0.065231 -0.362162   
416         -0.365546 -0.365546 -0.121276  3.814815 -0.073625 -0.370107   
417         -0.541935 -0.541935 -0.135324  0.662791 -0.104184 -0.568306   
418         -0.608511 -0.608511 -0.130418 -2.500000 -0.091407 -0.625483   
419         -0.510101 -0.510101 -0.128833  0.267606 -0.051728 -0.481013   

                             ARPU  
METRIC_NAME     TOTAL              
415         -0.117893  377.405175  
416         -0.116698  362.818478  
417         -0.132625  355.448584  
418         -0.126821  362.211613  
419         -0.120191  356.461272  

[5 rows x 70 columns]

## Analysis

In [75]:
''' Create Dataframe '''

df_revenue = df_mth[df_mth['METRIC_NAME']=='TOL Revenue']
df_revenue = df_revenue.rename(columns={'ACT':'REV'})

df_active_sub = df_mth[df_mth['METRIC_NAME']=='TOL Active Subs'][['TM_KEY_MTH', 'AREA_CD', 'ACT']]
df_active_sub = df_active_sub.rename(columns={'ACT':'ACTV_SUB'})

df_net_add = df_mth[df_mth['METRIC_NAME']=='TOL Net Add'][['TM_KEY_MTH', 'AREA_CD', 'ACT']]
df_net_add = df_net_add.rename(columns={'ACT':'NET_ADD'})

df_con_sub = df_mth[df_mth['METRIC_NAME']=='TOL Connected Subs'][['TM_KEY_MTH', 'AREA_CD', 'ACT']]
df_con_sub = df_con_sub.rename(columns={'ACT':'CON_SUB'})

df_chrn_sub = df_mth[df_mth['METRIC_NAME']=='TOL Churn Subs'][['TM_KEY_MTH', 'AREA_CD', 'ACT']]
df_chrn_sub = df_chrn_sub.rename(columns={'ACT':'CHRN_SUB'})

dfs = [df_revenue, df_active_sub, df_net_add, df_con_sub, df_chrn_sub]
df_raw_analysis = reduce(lambda left,right: pd.merge(left,right,on=['TM_KEY_MTH', 'AREA_CD']), dfs)
# df_raw_analysis = df_revenue.merge(df_active_sub, on=['TM_KEY_MTH', 'AREA_CD'])
df_raw_analysis['ARPU'] = df_raw_analysis['REV'] / df_raw_analysis['ACTV_SUB']
df_raw_analysis['MTH'] = 'M' + df_raw_analysis['TM_KEY_MTH'].apply(str).str[4:]

df_raw_analysis.head()

,METRIC_CD,METRIC_NAME,TM_KEY_MTH,MTH_DESC,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,AGG_TYPE,REV,...,GAP,LAST_ACT,DIFF,%MOM,ACTV_SUB,NET_ADD,CON_SUB,CHRN_SUB,ARPU,MTH
0,TB3R000100,TOL Revenue,202303,March,3,H,001,"BKK : Phra Nakhon, Dusit",S,6.506728e+06,...,-7.939695e+05,NaN,NaN,NaN,12945.0,9.0,142.0,133.0,502.644106,M03
1,TB3R000100,TOL Revenue,202303,March,3,H,003,"BKK : Nong Chok, Khlong Sam Wa",S,2.267262e+07,...,-2.815697e+06,NaN,NaN,NaN,45873.0,-283.0,244.0,527.0,494.247613,M03
2,TB3R000100,TOL Revenue,202303,March,3,H,004,"BKK : Bang Rak, Sathon",S,1.343712e+07,...,-8.953028e+05,NaN,NaN,NaN,24846.0,130.0,384.0,254.0,540.816383,M03
3,TB3R000100,TOL Revenue,202303,March,3,H,005,"BKK : Bang Khen, Lak Si",S,2.211108e+07,...,-2.628088e+06,NaN,NaN,NaN,44244.0,-162.0,295.0,457.0,499.753089,M03
4,TB3R000100,TOL Revenue,202303,March,3,H,006,"BKK : Bang Kapi, Saphan Sung",S,2.119239e+07,...,-2.461997e+06,NaN,NaN,NaN,42046.0,92.0,511.0,419.0,504.028690,M03


In [76]:
''' Pivot '''

df_analysis = df_raw_analysis.pivot(index=['AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME'], columns='MTH', 
                                    values=['REV', 'TGT', 'ACTV_SUB', 'ARPU', 'NET_ADD', 'CON_SUB', 'CHRN_SUB']).reset_index()
df_analysis = df_analysis.rename(columns={'REV':'REV_', 'TGT':'TGT_', 'ACTV_SUB':'ACTV_SUB_', 'ARPU':'ARPU_', 'CON_SUB':'CON_SUB_', 'CHRN_SUB':'CHRN_SUB_', 'NET_ADD':'NET_ADD_', 'CON_SUB':'CON_SUB_', 'CHRN_SUB':'CHRN_SUB_'})
df_analysis.columns = df_analysis.columns.map(''.join)

# df_analysis['PRODUCT'] = 'TOL'
df_analysis['REV_Q2'] = df_analysis['REV_M04'] + df_analysis['REV_M05'] + df_analysis['REV_M06']
df_analysis['TGT_Q2'] = df_analysis['TGT_M04'] + df_analysis['TGT_M05'] + df_analysis['TGT_M06']
df_analysis['ACTV_SUB_Q2'] = df_analysis['ACTV_SUB_M04'] + df_analysis['ACTV_SUB_M05'] + df_analysis['ACTV_SUB_M06']
df_analysis['CON_SUB_Q2'] = df_analysis['CON_SUB_M04'] + df_analysis['CON_SUB_M05'] + df_analysis['CON_SUB_M06']
df_analysis['CHRN_SUB_Q2'] = df_analysis['CHRN_SUB_M04'] + df_analysis['CHRN_SUB_M05'] + df_analysis['CHRN_SUB_M06']

df_analysis['ACHV_Q2'] = df_analysis['REV_Q2'] / df_analysis['TGT_Q2']
df_analysis['ARPU_Q2'] = df_analysis['REV_Q2'] / df_analysis['ACTV_SUB_Q2']
df_analysis['NET_ADD_Q2'] = df_analysis['CON_SUB_Q2'] - df_analysis['CHRN_SUB_Q2']

df_analysis.tail()

,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REV_M03,REV_M04,REV_M05,REV_M06,TGT_M03,TGT_M04,...,CHRN_SUB_M05,CHRN_SUB_M06,REV_Q2,TGT_Q2,ACTV_SUB_Q2,CON_SUB_Q2,CHRN_SUB_Q2,ACHV_Q2,ARPU_Q2,NET_ADD_Q2
100,3,H,92Y,Trang - Satun,6.127935e+06,6.272409e+06,6.258947e+06,5.521060e+06,6.487715e+06,6.559572e+06,...,368.0,352.0,1.805242e+07,1.917452e+07,44213.0,516.0,1032.0,0.941479,408.305620,-516.0
101,3,H,93Z,Phatthalung,5.154683e+06,5.321666e+06,5.315961e+06,4.695597e+06,4.943545e+06,5.020334e+06,...,267.0,281.0,1.533322e+07,1.474463e+07,39046.0,697.0,802.0,1.039919,392.696403,-105.0
102,3,H,94Z,Pattani,3.362055e+06,3.435746e+06,3.413618e+06,2.960887e+06,3.363869e+06,3.435215e+06,...,242.0,218.0,9.810251e+06,1.014914e+07,25281.0,382.0,650.0,0.966609,388.048371,-268.0
103,3,H,95Z,Yala,3.068429e+06,3.145654e+06,3.147649e+06,2.748462e+06,2.958237e+06,3.018572e+06,...,186.0,168.0,9.041765e+06,8.910668e+06,22906.0,481.0,550.0,1.014712,394.733458,-69.0
104,3,H,96Z,Narathiwat,3.275176e+06,3.309994e+06,3.363215e+06,2.958985e+06,3.316640e+06,3.375809e+06,...,269.0,224.0,9.632194e+06,9.938724e+06,25136.0,481.0,721.0,0.969158,383.203139,-240.0


In [77]:
''' Find Bottom 20 Province '''

# df_rev_analysis['1MTH_DROP'] = np.where(df_rev_analysis['M_04'] > df_rev_analysis['M_05'], 'Y', 'N')
# df_rev_analysis['2MTH_DROP'] = np.where((df_rev_analysis['1MTH_DROP']=='Y') & (df_rev_analysis['M_05'] > df_rev_analysis['M_06']), 'Y', 'N')
df_analysis['2MTH_DROP'] = np.where((df_analysis['REV_M04'] > df_analysis['REV_M05']) & (df_analysis['REV_M05'] > df_analysis['REV_M06']), 'Y', 'N')
df_analysis = df_analysis[(df_analysis['2MTH_DROP']=='Y') & (df_analysis['AREA_TYPE']=='H')]

df_analysis['GROWTH'] = df_analysis['REV_M06'] / df_analysis['REV_M04']
df_analysis['RNK_NO_GROWTH'] = df_analysis.groupby('AREA_TYPE')['GROWTH'].rank().astype(int)
df_analysis = df_analysis[df_analysis['RNK_NO_GROWTH'] <= 20]

df_analysis['REV_GAP'] = df_analysis['REV_M06'] - df_analysis['REV_M04']
df_analysis['RNK_REV_GAP'] = df_analysis.groupby('AREA_TYPE')['REV_GAP'].rank().astype(int)

df_analysis = df_analysis.reset_index(drop=True)
df_analysis = df_analysis.sort_values('REV_GAP')

# df_analysis.sort_values('RNK_NO_GROWTH')
df_analysis

# ''' Get results '''
# df_analysis.to_excel('df_analysis.xlsx', index=False)

,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,REV_M03,REV_M04,REV_M05,REV_M06,TGT_M03,TGT_M04,...,CON_SUB_Q2,CHRN_SUB_Q2,ACHV_Q2,ARPU_Q2,NET_ADD_Q2,2MTH_DROP,GROWTH,RNK_NO_GROWTH,REV_GAP,RNK_REV_GAP
7,3,H,043,"BKK : Min Buri, Khan Na Yao, Bueng Kum",2.726084e+07,2.746172e+07,2.745483e+07,2.392289e+07,3.086166e+07,3.078370e+07,...,2299.0,1545.0,0.889226,481.900685,754.0,Y,0.871136,10,-3.538830e+06,1
8,3,H,050,"BKK : Bang Khun Thian, Chom Thong, Bang Bon",2.650026e+07,2.670935e+07,2.662762e+07,2.339166e+07,2.974019e+07,2.968951e+07,...,1590.0,1714.0,0.896494,473.461054,-124.0,Y,0.875786,20,-3.317684e+06,2
9,3,H,103,SMP : Bang Phli,2.328806e+07,2.361298e+07,2.359547e+07,2.059767e+07,2.559769e+07,2.560994e+07,...,1265.0,1295.0,0.916229,502.371776,-30.0,Y,0.872303,14,-3.015308e+06,3
6,3,H,036,"BKK : Don Mueang, Sai Mai",2.259347e+07,2.282842e+07,2.281470e+07,1.996719e+07,2.514395e+07,2.512924e+07,...,414.0,1481.0,0.904580,478.530123,-1067.0,Y,0.874664,17,-2.861229e+06,4
4,3,H,023,"BKK : Nong Khaem, Bang Khae",2.039211e+07,2.052699e+07,2.044804e+07,1.788568e+07,2.291008e+07,2.290444e+07,...,785.0,1274.0,0.890015,464.113271,-489.0,Y,0.871325,11,-2.641308e+06,5
3,3,H,016,"BKK : Bangkok Yai, Bangkok Noi, Bang Phlat",1.709884e+07,1.724755e+07,1.719675e+07,1.497596e+07,1.894268e+07,1.895427e+07,...,1257.0,1091.0,0.902148,466.849871,166.0,Y,0.868295,6,-2.271594e+06,6
5,3,H,024,"BKK : Rat Burana, Thung Khru",1.331514e+07,1.345232e+07,1.340415e+07,1.167732e+07,1.502138e+07,1.500248e+07,...,305.0,825.0,0.890547,470.981176,-520.0,Y,0.868052,5,-1.775001e+06,7
10,3,H,303,"PTT : Thanyaburi, Nong Suea",1.358165e+07,1.370153e+07,1.368376e+07,1.199223e+07,1.526341e+07,1.528369e+07,...,1479.0,1095.0,0.890753,473.691767,384.0,Y,0.875247,19,-1.709302e+06,8
2,3,H,015,"BKK : Thon Buri, Khlong San",1.207799e+07,1.230011e+07,1.225188e+07,1.072899e+07,1.307113e+07,1.307651e+07,...,817.0,699.0,0.933737,471.406108,118.0,Y,0.872268,12,-1.571119e+06,9
1,3,H,012,"BKK : Yan Nawa, Bang Kho Laem",1.167203e+07,1.175079e+07,1.169865e+07,1.020775e+07,1.298711e+07,1.298415e+07,...,1004.0,642.0,0.897732,485.827929,362.0,Y,0.868687,7,-1.543037e+06,10
